In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import tensorflow as tf

import atecml.data

from contextlib import contextmanager
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegressionCV
#build Models...
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE, ADASYN
import random

plt.style.use('ggplot')

In [2]:
train_df = pd.read_pickle('./01_train.dat')
#train_df = atecml.data.load_train()

predictors = [x for x in train_df.columns if x not in atecml.data.NOT_FEATURE_COLUMNS2]
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-13').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-14',end_date='2017-09-22').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-23',end_date='2017-10-01').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-10-02',end_date='2017-10-12').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-13',end_date='2017-10-22').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-23',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(765667, 229064)

In [3]:
WOE_LIST = joblib.load('./woe_feature.dat')
CATE_LIST = WOE_LIST + atecml.data.CATE_FEATURE_LIST

#CATE_LIST = atecml.data.CATE_FEATURE_LIST
categorical=[]
for item in predictors:
    if (item in CATE_LIST):
        categorical.append(item)


In [4]:
num_boosting_round = 3000
early_stop_round = 100

In [5]:
params = {
    
    'objective': 'binary',
    'metric': 'auc',
    'use_missing' : True,
    'is_unbalance': True,
    #'scale_pos_weight': 98,
    'learning_rate': 0.05,
    'num_leaves': 64,  # we should let it be smaller than 2^(max_depth)
    'max_depth': -1,  # -1 means no limit
    'min_child_samples': 600,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 255,  # Number of bucketed bin for feature values
    'colsample_bytree': 0.4,
    'subsample': 0.85,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'min_child_weight': 0.05,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'subsample_for_bin': 200000,  # Number of samples for constructing bin
    'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
    'reg_alpha': 0.01,  # L1 regularization term on weights
    'reg_lambda': 0.1,  # L2 regularization term on weights
    'nthread': 40,
    'n_estimators': num_boosting_round,
    'verbose': -1,
}

rf = {
    'boosting_type': 'rf',
}
rf.update(params)

dart = {
    'boosting_type': 'dart',
}
dart.update(params)

gbdt = {
    'boosting_type': 'gbdt',
}
gbdt.update(params)

param_list = [gbdt,dart]

In [ ]:
model_list  =[]

for idx in tqdm(range(0,15)):
    fold_id = idx//3
    Train_DataSet = train_df[train_df.index.isin(list(all_list - DateFold[fold_id]))].reset_index(drop=True)
    Normal_DF = Train_DataSet[Train_DataSet['label']==0]
    Fraud_DF = Train_DataSet[Train_DataSet['label']==1]
    
    number_record_fraud = len(Fraud_DF)
    number_record_normal = len(Normal_DF)
    #undersample
    random_normal_indices = np.array(np.random.choice(Normal_DF.index,number_record_fraud,replace=False))
    filter_list = list(random_normal_indices) + list(Fraud_DF.index)
    under_sample_train = Train_DataSet[Train_DataSet.index.isin(filter_list)].reset_index(drop=True)
    
    Val_DataSet = train_df[train_df.index.isin(DateFold[idx])].reset_index(drop=True)
    X_train = under_sample_train[predictors]
    y_train = under_sample_train['Fraud']

    X_test = Val_DataSet[predictors]
    y_test = Val_DataSet['Fraud']
    
    for item_params in (param_list):
        gbm = lgb.LGBMClassifier(**item_params)
        print('starting fit model...')
        gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)],eval_metric='auc',early_stopping_rounds=early_stop_round,verbose=200)#,categorical_feature=categorical)
        model_list.append(gbm)  

  0%|          | 0/15 [00:00<?, ?it/s]

starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.945743
[400]	valid_0's auc: 0.949167
[600]	valid_0's auc: 0.949265
Early stopping, best iteration is:
[578]	valid_0's auc: 0.949526
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.927879
[400]	valid_0's auc: 0.943937
[600]	valid_0's auc: 0.947421
[800]	valid_0's auc: 0.949348
[1000]	valid_0's auc: 0.95029
[1200]	valid_0's auc: 0.95105
[1400]	valid_0's auc: 0.951452


  7%|▋         | 1/15 [01:33<21:44, 93.17s/it]

Early stopping, best iteration is:
[1363]	valid_0's auc: 0.95159
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.976605
[400]	valid_0's auc: 0.982568
[600]	valid_0's auc: 0.985032
[800]	valid_0's auc: 0.986186
[1000]	valid_0's auc: 0.986792
[1200]	valid_0's auc: 0.987327
[1400]	valid_0's auc: 0.987733
[1600]	valid_0's auc: 0.988083
[1800]	valid_0's auc: 0.988363
[2000]	valid_0's auc: 0.98861
[2200]	valid_0's auc: 0.988821
[2400]	valid_0's auc: 0.98897
[2600]	valid_0's auc: 0.989093
[2800]	valid_0's auc: 0.989179
[3000]	valid_0's auc: 0.989333
Did not meet early stopping. Best iteration is:
[3000]	valid_0's auc: 0.989333
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.964536
[400]	valid_0's auc: 0.973693
[600]	valid_0's auc: 0.977704
[800]	valid_0's auc: 0.980106
[1000]	valid_0's auc: 0.981827
[1200]	valid_0's auc: 0.982887
[1400]	valid_0's auc: 0.983846
[1600]	valid_0

 13%|█▎        | 2/15 [05:07<33:21, 153.93s/it]

...

[3000]	valid_0's auc: 0.988567
Did not meet early stopping. Best iteration is:
[2988]	valid_0's auc: 0.98857
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.981
[400]	valid_0's auc: 0.985186
[600]	valid_0's auc: 0.987237
[800]	valid_0's auc: 0.988354
[1000]	valid_0's auc: 0.989029
[1200]	valid_0's auc: 0.98947
[1400]	valid_0's auc: 0.989763
[1600]	valid_0's auc: 0.989926
[1800]	valid_0's auc: 0.990088
[2000]	valid_0's auc: 0.990242
[2200]	valid_0's auc: 0.990358
[2400]	valid_0's auc: 0.99044
[2600]	valid_0's auc: 0.990485
[2800]	valid_0's auc: 0.990562
[3000]	valid_0's auc: 0.990617
Did not meet early stopping. Best iteration is:
[2974]	valid_0's auc: 0.990636
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.970222
[400]	valid_0's auc: 0.978294
[600]	valid_0's auc: 0.981783
[800]	valid_0's auc: 0.983356
[1000]	valid_0's auc: 0.984634
[1200]	valid_0's auc: 0.985454
[140

 27%|██▋       | 4/15 [12:57<35:38, 194.45s/it]

[3000]	valid_0's auc: 0.989
Did not meet early stopping. Best iteration is:
[3000]	valid_0's auc: 0.989
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.979027
[400]	valid_0's auc: 0.984717
[600]	valid_0's auc: 0.986864
[800]	valid_0's auc: 0.988089
[1000]	valid_0's auc: 0.98888
[1200]	valid_0's auc: 0.989414
[1400]	valid_0's auc: 0.989803
[1600]	valid_0's auc: 0.990064
[1800]	valid_0's auc: 0.990283
[2000]	valid_0's auc: 0.99049
[2200]	valid_0's auc: 0.990621
[2400]	valid_0's auc: 0.990755
[2600]	valid_0's auc: 0.990892
Early stopping, best iteration is:
[2690]	valid_0's auc: 0.99095
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.963264
[400]	valid_0's auc: 0.974869
[600]	valid_0's auc: 0.979572
[800]	valid_0's auc: 0.981827
[1000]	valid_0's auc: 0.983534
[1200]	valid_0's auc: 0.984524
[1400]	valid_0's auc: 0.98534
[1600]	valid_0's auc: 0.986003
[1800]	valid_0's auc:

 33%|███▎      | 5/15 [16:57<33:55, 203.58s/it]

[3000]	valid_0's auc: 0.988881
Did not meet early stopping. Best iteration is:
[2998]	valid_0's auc: 0.988892
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.965231
[400]	valid_0's auc: 0.966899
Early stopping, best iteration is:
[401]	valid_0's auc: 0.966925
starting fit model...
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.958979
[400]	valid_0's auc: 0.963801
[600]	valid_0's auc: 0.965406
[800]	valid_0's auc: 0.966111


In [ ]:
val_df = train_df[train_df.index.isin(DateFold[5])].reset_index(drop=True)
val_df1 = val_df.head(180000)
val_df2 = val_df[~val_df.index.isin(list(val_df1.index))]

In [ ]:
val_r_neg=pd.DataFrame()
for idx in tqdm(range(0,len(model_list))):
    model_neg = model_list[idx]
    val_neg = model_neg.predict_proba(val_df[predictors])
    val_r_neg[idx] = pd.DataFrame(val_neg)[1]
    
    

In [ ]:
#pos_mean = val_r_pos.mean(axis=1)
neg_mean = val_r_neg.mean(axis=1)
_,_,_ = atecml.data.accuracy_validation(val_df['Fraud'],neg_mean)

In [ ]:
def stack_result(X_train,model_list):
    with atecml.data.timer('Classification: Building Layer-1 Stack'):
        input_list = []
        for idx in tqdm(range(len(model_list))):
            model = model_list[idx]
            _temp_df = pd.DataFrame(model.predict_proba(X_train))[1]
            input_list.append(pd.DataFrame(_temp_df))
        input_predict= np.array(pd.concat(input_list,ignore_index=True,axis=1))
    return input_predict




In [ ]:
level2_model_list  =[]
# use XGBOOST  as the model of the second layer
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


for idx in range(0,5):
    
    Train_DataSet = train_df[train_df.index.isin(list(all_list - DateFold[idx]))].reset_index(drop=True)
    Normal_DF = Train_DataSet[Train_DataSet['label']==0]
    Fraud_DF = Train_DataSet[Train_DataSet['label']==1]
    
    number_record_fraud = len(Fraud_DF)
    number_record_normal = len(Normal_DF)
    #undersample
    random_normal_indices = np.array(np.random.choice(Normal_DF.index,number_record_fraud,replace=False))
    filter_list = list(random_normal_indices) + list(Fraud_DF.index)
    under_sample_train = Train_DataSet[Train_DataSet.index.isin(filter_list)].reset_index(drop=True)
    
    X_train = Train_DataSet[predictors]
    y_train = Train_DataSet['Fraud']
    
    model = XGBClassifier(
                         learning_rate =0.05,
                         n_estimators=400,
                         max_depth=3,
                         min_child_weight=1,
                         gamma=0,
                         subsample=0.8,
                         colsample_bytree=0.9,
                         objective= 'binary:logistic',
                         scoring='roc_auc',
                         scale_pos_weight= 98,
                         nthread=40,
                         seed=27)
    
    X_train_level2 = stack_result(X_train,model_list)
    model.fit(X_train_level2,y_train)  
    level2_model_list.append(model)
        

In [ ]:
val_df2_level2 = stack_result(val_df[predictors],model_list)
foo = stack_result(val_df2_level2,level2_model_list)


In [ ]:
y_pred= pd.DataFrame(foo).min(axis=1)
_,_,_ = atecml.data.accuracy_validation(val_df['Fraud'],y_pred)

In [ ]:
test_df = atecml.data.load_test()

In [ ]:
test_df_level1 = stack_result(test_df[predictors],model_list)

In [ ]:
test_df_level2 = stack_result(test_df_level1,level2_model_list)

In [ ]:
y_pred= pd.DataFrame(test_df_level2).min(axis=1)

In [ ]:
result=pd.DataFrame()
result['id'] = test_df['id']
result['score'] = y_pred

In [ ]:
result.to_csv('./0706_03.csv',index=False)

In [ ]:
unknown_df =atecml.data.load_train()

In [ ]:
unknown_df = unknown_df[unknown_df.label==-1].reset_index(drop=True)

In [ ]:
un_level1 = stack_result(unknown_df[predictors],model_list)
un_level2 = stack_result(un_level1,level2_model_list)

In [ ]:
un_pred= pd.DataFrame(un_level2).min(axis=1)

In [ ]:
un_pred.hist(bins=100)

In [ ]:
unresult=pd.DataFrame()
unresult['id'] = unknown_df['id']
unresult['score'] = un_pred

In [ ]:
unresult.to_pickle('./reject_inf.dat')